# 0 Imports

In [11]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scipy.io
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact

In [12]:
# path para guardar o dataset
PATH = './'
PATH_TRAIN_CSV = './data/train.csv'
PATH_TRAIN_IMG = './data/train_data.mat'
PATH_TEST_CSV = './data/test.csv'
PATH_TEST_IMG = './data/test_data.mat'

# 1 Exploração e Preparação dos Dados

### Visualização dos dados

In [13]:
def get_data_from_mat(train_file,test_file):
    train_mat = scipy.io.loadmat(train_file) 
    test_mat = scipy.io.loadmat(test_file) 
    # print(train_mat.keys())
    # print(test_mat.keys())
    train_np = np.array(train_mat['train_data']).transpose(2,0,1)
    test_np = np.array(test_mat['test_data']).transpose(2,0,1)
    # print(train_np.shape)
    # print(test_np.shape)
    return  train_np, test_np


def load_data(path_train_csv, path_train_img, path_test_csv, path_test_img):
    train_csv = pd.read_csv(path_train_csv, header=0)
    test_csv = pd.read_csv(path_test_csv, header=0)
    train_img, test_img = get_data_from_mat(path_train_img, path_test_img)
    # train = train_csv + train_img
    # test = test_csv + test_img
    # return train, test
    return train_csv, test_csv, train_img, test_img


def visualize(image):
    #plt.figure("sample", (12, 6))
    #plt.subplot(1, 2, 1)
    plt.imshow(image, cmap="gray")    
    #plt.subplot(1, 2, 2)
    #plt.imshow(image, cmap="gray")
    plt.show()      

def show_ds(ds):
    print("ds shape:",ds.shape)
    print("ds max:",np.max(ds))
    print("ds min:",np.min(ds))
    print("ds average:",np.average(ds))
    @interact
    def visualize_set(scan_index=(0,len(ds)-1)):
        #print(scan_index)
        visualize(ds[scan_index,:,:])

In [14]:
train_csv, test_csv, train_img, test_img = load_data(PATH_TRAIN_CSV, PATH_TRAIN_IMG, PATH_TEST_CSV, PATH_TEST_IMG)

print("Data de treino:")
print(train_csv)
show_ds(train_img)
print("Data de Teste:")
print(test_csv)
show_ds(test_img)

Data de treino:
      id  age  sex  education
0      1   13    1          7
1      2   14    0          8
2      3   15    1          9
3      4   15    1          9
4      5   15    1          9
..   ...  ...  ...        ...
107  108   77    1          4
108  109   67    0          4
109  110   55    0          4
110  111   76    1          3
111  112   69    0          4

[112 rows x 4 columns]
ds shape: (112, 90, 90)
ds max: 1.0
ds min: 0.0
ds average: 0.024116160549305543


interactive(children=(IntSlider(value=55, description='scan_index', max=111), Output()), _dom_classes=('widget…

Data de Teste:
    id  sex  education
0    1    0         13
1    2    0         11
2    3    1          9
3    4    1         13
4    5    0         12
5    6    0         17
6    7    0          9
7    8    0          4
8    9    1          9
9   10    1          4
10  11    1         14
11  12    0          9
12  13    1          2
13  14    1          5
14  15    1          9
15  16    1         11
16  17    1          9
17  18    0         17
18  19    1         15
19  20    1         14
20  21    1          4
21  22    0          4
22  23    1          4
23  24    0          0
24  25    1          2
25  26    1          4
26  27    1          3
27  28    1          4
ds shape: (28, 90, 90)
ds max: 1.0
ds min: 0.0
ds average: 0.024994026906171023


interactive(children=(IntSlider(value=13, description='scan_index', max=27), Output()), _dom_classes=('widget-…

### Preparação dos dados

In [15]:
def fix_sex(csv):
    female = csv['sex']
    male = []
    for person in female:
        if person==1:
            male.append(0)
        else:
            male.append(1)
    csv.drop('sex', axis='columns', inplace=True)
    csv['Female']=female
    csv['Male']=male
    csv.drop('id', axis='columns', inplace=True)
    return csv
    
def img_to_list(img):
    tamanho = len(img)
    lista = []
    # triangular inferior sem diagonal
    for linha in range(tamanho):
        for coluna in range(linha):
            lista.append(img[linha][coluna])
    return lista

def imgs_to_matrix(imgs):
    matrix = []
    for img in imgs:
        matrix.append(img_to_list(img))
    return matrix

def remove_null_columns(matrix1, matrix2):
    columns_to_remove = []
    m1_row_len = len(matrix1)
    m2_row_len = len(matrix2)
    column_len = len(matrix1[0])
    for column in range(column_len):
        all_zero = True
        for row in range(m1_row_len):
            if matrix1[row][column]!=0:
                all_zero=False
        for row in range(m2_row_len):
            if matrix2[row][column]!=0:
                all_zero=False
        if all_zero:
            columns_to_remove.append(column)

    brain_activity_index = list(range(column_len))
    for column in columns_to_remove[::-1]:
        brain_activity_index.pop(column)
        for line in range(m1_row_len):
            matrix1[line].pop(column)
        for line in range(m2_row_len):
            matrix2[line].pop(column)
    return matrix1, matrix2, brain_activity_index

def join_data(train_csv, test_csv, train_img, test_img):
    train_matrix = imgs_to_matrix(train_img)
    test_matrix = imgs_to_matrix(test_img)
    train_clean_matrix, test_clean_matrix, brain_activity_index = remove_null_columns(train_matrix, test_matrix)
    train_data  = pd.concat([train_csv, pd.DataFrame(train_clean_matrix)], axis=1)
    train_data .columns = list(train_csv.columns) + [f'rel-{brain_activity_index[i]}' for i in range(len(brain_activity_index))]
    test_data = pd.concat([test_csv, pd.DataFrame(test_clean_matrix)], axis=1)
    test_data.columns = list(test_csv.columns) + [f'rel-{brain_activity_index[i]}' for i in range(len(brain_activity_index))]
    return train_data , test_data, brain_activity_index

In [16]:
train_csv = fix_sex(train_csv)
test_csv = fix_sex(test_csv)
train_data, test_data, brain_activity_index = join_data(train_csv, test_csv, train_img, test_img)
print(train_data)
print(test_data)

     age  education  Female  Male     rel-1  rel-3     rel-4     rel-5   
0     13          7       1     0  0.131525    0.0  0.136950  0.000000  \
1     14          8       0     1  0.117704    0.0  0.132998  0.037714   
2     15          9       1     0  0.155806    0.0  0.165429  0.022278   
3     15          9       1     0  0.145214    0.0  0.162493  0.000000   
4     15          9       1     0  0.161360    0.0  0.172043  0.017686   
..   ...        ...     ...   ...       ...    ...       ...       ...   
107   77          4       1     0  0.215767    0.0  0.184942  0.019446   
108   67          4       0     1  0.143005    0.0  0.124350  0.000000   
109   55          4       0     1  0.116085    0.0  0.156406  0.020947   
110   76          3       1     0  0.156654    0.0  0.166613  0.046464   
111   69          4       0     1  0.114586    0.0  0.128877  0.045583   

        rel-8    rel-13  ...  rel-3975  rel-3977  rel-3979  rel-3981   
0    0.146309  0.085044  ...  0.000000 

In [20]:
def label(data):
    label_data = train_data['age']
    train_data.drop('age', axis='columns', inplace=True)
    return train_data, label_data

train_data, label_data = label(train_data)
print(train_data)

KeyError: 'age'

# 2 Definir o Modelo

In [29]:
# Create the Random Forest classifier
rf_model = RandomForestClassifier()

# Create the XGBoost classifier
xgb_model = XGBClassifier()

# 3 Treinar o Modelo

In [22]:
# Fit the model to the training data
rf_model.fit(train_data, label_data)

RandomForestClassifier()

In [32]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(label_data)

# Fit the model to the training data
xgb_model.fit(train_data, y_encoded)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

# 4 Avaliar o Modelo

# 5 Testar o Modelo

In [28]:
# Random Forest predictions
rf_predictions = rf_model.predict(test_data)
print(rf_predictions)

[16 15 16 16 18 25 76 69 76 63 69 69 69 62 16 16 16 15 18 15 68 58 69 69
 69 76 55 69]


In [36]:
# XGBoost predictions
xgb_predictions = xgb_model.predict(test_data)
y_pred_original = label_encoder.inverse_transform(xgb_predictions)

print(y_pred_original)

[15 16 16 16 16 25 69 69 64 25 63 63 63 55 16 15 16 24 24 46 68 69 56 55
 69 56 63 76]
